In [8]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

In [4]:
model = AutoModelForQuestionAnswering.from_pretrained('./models/test_qa_model')
tokenizer = AutoTokenizer.from_pretrained('./models/test_qa_model')


In [5]:
def answer_question(question, context):
    inputs = tokenizer.encode_plus(
        question,
        context,
        add_special_tokens=True,
        return_tensors="pt",
        max_length=512,
        truncation=True,
    )
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits) + 1

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0][start_index:end_index])

    answer = tokenizer.convert_tokens_to_string(tokens)

    return answer.strip()


In [6]:
from datasets import load_dataset

squad_dataset = load_dataset('squad')

In [14]:
# Select a sample from the validation set
sample = squad_dataset['validation'][0]
question = sample['question']
context = sample['context']

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")
print(f"True Answer: {sample['answers']}")

Question: Which NFL team represented the AFC at Super Bowl 50?
Answer: Carolina Panthers
True Answer: {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answer_start': [177, 177, 177]}


In [15]:
print(context)

Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.


In [12]:
# Custom context and question
context = """
The Apollo program was the third United States human spaceflight program carried out by the National Aeronautics and Space Administration (NASA),
which succeeded in landing the first humans on the Moon from 1969 to 1972.
"""
question = "Who landed the first humans on the Moon?"

print(f"Question: {question}")
print(f"Answer: {answer_question(question, context)}")

Question: Who landed the first humans on the Moon?
Answer: 1969 to 1972
